# Lecture 06 – Maps

## Data 6, Visualizations Module

In [ ]:
from datascience import *
import numpy as np
import plotly.express as px

Table.interactive_plots()

The first dataset today comes from [Kaggle](https://www.kaggle.com/datasets/datafiniti/fast-food-restaurants/versions/1?resource=download) (which in turn got the data from [Datafinit's](https://datafiniti.co/products/business-data/) Business Database) and shows the locations of over 10,000 fast food restaurants across the country. There are over 500 unique fast food chains represented in this data, and the geographical data makes it perfect for mapping.

In [ ]:
restaurants = Table.read_table('data/FastFoodRestaurants.csv').select('name', 'city', 'latitude', 'longitude', 'province', 'country')
restaurants

## Review: Scatter Plots, Grouping

Before we start creating maps today, it may be helpful to go over a quick review of two important building blocks for maps: scatter plots and `.group`.

### Scatter Plots

Scatter plots are helpful for visualizing two numerical variables. Often, these two variables represent two different coordinates (like x and y or latitude and longitude). This makes scatter plots very well suited for visualizing **spatial data**. We can generate a scatter plot by calling `tbl.scatter()`

In [ ]:
restaurants.where('longitude', are.below(-50)).scatter('longitude', 'latitude')

Does this scatter plot resemble anything?

As we can see above, scatter plots are useful for visualizing geographic data, but it would be even more helpful if we could overlay this geographic data on top of a recognizable map. This will help maintain the right dimesions and proportions.

### Grouping

Another important table method we will use while mapping is `tbl.group()`. `.group` is really powerful because it can help us aggregate our data to visualize it at a higher **unit of analysis** (e.g. at the state or country level).

Recall that `tbl.group(col)` will group the table on the column `col`, generating a table with all the unique values in `col` as the rows, and an additional column with the counts for each of these values.

In [ ]:
restaurants.group('name').sort('count', descending=True)

Using `.group` we can find the fast food chains with the most locations across the US.

We can also use `.group` to find the most common fast food chain within each state. The implementation of this is a little complicated, so we have done it for you. You can use `most_common_by_state(tbl)` to generate a table of the most common restaurant for each state.

In [ ]:
def first_item(arr):
    return arr.item(0)

def most_common_by_state(tbl):
    """Returns a table with the count of the most common restaurant chain for each state in tbl"""
    by_state = tbl.group(make_array('province', 'name')).sort('count', descending = True) # Group by state and chain
    most_common = by_state.group('province', collect = first_item) # Group by state, finding most common chain
    return most_common.relabeled('name first_item', 'name').relabeled('count first_item', 'count') # Relabel columns

In [ ]:
favorite_restaurant = most_common_by_state(restaurants)
favorite_restaurant

## Scatter Plot Maps

### Scatter Plot + Map = ❤️

When we want to visualize the geographic locations of a lot of data points, it's often helpful to start with a scatter plot map. Scatter plot maps are exactly like scatter plots, with the addition of a geographic map behind them to help you visualize geographic locations in relation to cities, states, and countries.

In [ ]:
# This code will get us only the restaurants from the top 5 chains nationally
top_5_restaurant_names = restaurants.group('name').sort('count', descending=True).take(np.arange(5)).column('name')

top_5_restaurants = restaurants.where('name', are.contained_in(top_5_restaurant_names))
top_5_restaurants

For example, we can visualize the locations of restaurants from the top 5 restaurant chains nationally.

### Fast Food Chain Locations

In [ ]:
# Just run me
px.scatter_geo(top_5_restaurants.to_df(), 
               lat = 'latitude', 
               lon = 'longitude', 
               color = 'name',
               locationmode='USA-states',
               scope = 'usa',
               title = "Top 5 Fast Food Chain Locations"
              )

Here, we are using a Python library called **Plotly** (technically, the library is called Plotly Express - hence `px`). Plotly is a commonly used visualization library and is really useful for creating maps.

However, the Plotly library can be quite confusing and difficult to fully wrap your head around. Even the instructors regularly rely on the [Plotly documentation](https://plotly.com/python/plotly-express/) to create and debug visualizations. For this lecture, we will provide a lot of Plotly skeleton to help you create visualizations. Plotly is nice partly because it is easy to copy and paste code from the documentation, changing it to suit your needs.

**Note: we do not expect you to memorize/remember any Plotly syntax. This is purely for fun/learning — the Plotly library is out of scope for Data 6.**

### Now, let's figure out what's going on

Plotly has a lot of its own functions that you can use to generate visualizations. The function we are using to create our scatter plot maps is `px.scatter_geo()`. This function takes a few mandatory as well as many optional arguments:
* `df`: The first argument is the _data frame_ (df) you want to use as your data. For Data 6 purposes, _data frame_ is the same as a table, but we will need to convert our tables to data frames using `tbl.to_df()`


* `lat`: This argument specifies which column in `df` (our table) to use as the latitude (pass in a column name string)


* `lon`: This argument specifies which column in `df` (our table) to use as the longitude (pass in a column name string)


* `color`: The `color` argument is the column we want to use to generate different colors (categories) for our points. It is very similar to the `group` argument in `tbl.hist` and `tbl.scatter`


* `locationmode` and `scope`: These arguments specify how the map will display. `locationmode='USA-states'` and `scope='usa'` will give us a state map of the US and no other country


* `title`: The title for our table!

Now that we have a better sense of what's going on, let's try generating some more scatter plot maps

In [ ]:
px.scatter_geo(restaurants.to_df(), 
               lat = 'latitude', 
               lon = 'longitude', 
               color = 'name',
               locationmode='USA-states',
               scope = 'usa',
               title = 'All Restaurants in Our Dataset'
              )

We _can_ plot all 10000+ restaurants in our table, but it gets really messy and hard to interpret. Again, just because you _can_ do something doesn't mean you _should_. Also, when it comes to visualization, oftentimes **less is more**.

### Regional Fast Food Chains

Another fun thing we could do with scatter plot maps is look at regional differences in fast food chains.

In [ ]:
regional_chains = make_array('Waffle House', 'In-N-Out Burger', "Taco John's", "Whataburger", "Dunkin' Donuts")

Here, we have specified a subset of 5 regional fast food chains local to different parts of the US. We can use the `are.contained_in()` predicate with `tbl.where` in order to only get restaurants from these chains.

In [ ]:
regional_restaurants = restaurants.where('name', are.contained_in(regional_chains))
regional_restaurants

Now, let's see where these restaurants tend to be located.

In [ ]:
px.scatter_geo(regional_restaurants.to_df(), 
               lat = 'latitude', 
               lon = 'longitude', 
               color = 'name',
               locationmode='USA-states',
               scope = 'usa',
               title = 'Regional Fast Food Chains Across the US'
              )

As a side note: this dataset seems to be fairly outdated/limited. This is evident in the lack of Dunkin' Donuts nationwide,

## Choropleth Maps

Choropleth maps are useful for visualizing numerical variables across different states or countries. In this sense they are analogous to bar charts, since they encode one categorical variable (state or country) and one numerical variable.

Using `px.choropleth` we can visualize the most common fast food chain for each state. Recall our `favorite_restaurant` table from earlier:

In [ ]:
favorite_restaurant

In [ ]:
px.choropleth(favorite_restaurant.to_df(), 
              locations = 'province', 
              color = 'name',
              color_discrete_sequence = px.colors.qualitative.Bold,
              locationmode='USA-states',
              scope="usa",
              title = "Most Common Fast Food Chain by State"
             )

`px.choropleth()` takes similar arguments to `px.scatter_geo()`, but there are a few worth noting here:
* `df`: `px.choropleth`, like all other Plotly express functions, needs a data frame (a table to work with). Don't forget to convert your table to a data frame using `tbl.to_df()`


* `locations`: This argument specifies the column of the locations for each numerical value. If you are plotting states, this column *must* contain the two-letter state abbreviations (e.g. 'CA' for California). If you are plotting countries, the column must contain the three-letter ISO country code (more on this later).


* `color`: This specifies the argument with different categories to color by


* `color_discrete_sequence`: This argument specifies which color palette to use for the visualization. Plotly provides built-in color palettes with `px.colors.qualitative`. Feel free to play around with different color schemes (you can see more [here](https://plotly.com/python/discrete-color/))

Now, let's play around with our data to see what other visualizations we can make with choropleth maps.

How about the most common pizza chain by state:

In [ ]:
pizza_restaurants = restaurants.where('name', are.containing('Pizza'))
pizza_restaurants

### Favorite Pizza Chains

In [ ]:
pizza_by_state = most_common_by_state(pizza_restaurants)
pizza_by_state

In [ ]:
px.choropleth(pizza_by_state.to_df(), 
              locations = 'province', 
              color = 'name',
              color_discrete_sequence = px.colors.qualitative.Set2,
              locationmode='USA-states',
              scope="usa",
              hover_name = 'count',
              title = "Most Common Pizza Chain by State",
             )

### McDonald's vs. Burger King

What's more popular in each state: McDonald's or Burger King?

In [ ]:
bk_mcd = make_array("Burger King", "McDonald's")
bk_or_mcd_restaurants = restaurants.where('name', are.contained_in(bk_mcd))

In [ ]:
bk_mcd_favorite = most_common_by_state(bk_or_mcd_restaurants)

In [ ]:
px.choropleth(bk_mcd_favorite.to_df(), 
              locations = 'province', 
              color = 'name',
              color_discrete_sequence = px.colors.qualitative.Safe,
              locationmode='USA-states',
              scope="usa",
              hover_name = 'count',
              title = "Does Your State Have More McDonald's or Burger Kings?",
             )

### More Burgers!

How about burger chains more generally?

In [ ]:
burger_restaurants = restaurants.where('name', are.containing('Burger'))
burger_restaurants

In [ ]:
burgers_by_state = most_common_by_state(burger_restaurants)

In [ ]:
px.choropleth(burgers_by_state.to_df(), 
              locations = 'province', 
              color = 'name',
              color_discrete_sequence = px.colors.qualitative.Pastel,
              locationmode='USA-states',
              scope="usa",
              hover_name = 'count',
              title="Burger Kings Dominates 'Burger' Market"
             )

Oops! This map doesn't tell us much, since Burger King has the most restaurants in each state out of all chains with the word 'burger' in them. Let's exclude Burger King and see what happens:

In [ ]:
burger_restaurants_no_bk = burger_restaurants.where('name', are.not_equal_to('Burger King'))
burger_restaurants_no_bk

In [ ]:
most_common_burger = most_common_by_state(burger_restaurants_no_bk)

In [ ]:
px.choropleth(most_common_burger.to_df(), 
              locations = 'province', 
              color = 'name',
              color_discrete_sequence = px.colors.qualitative.Pastel,
              locationmode='USA-states',
              scope="usa",
              hover_name = 'count',
              title="Other Burger Chains in the US (Excludes McDonald's, Burger King, and Whataburger)"
             )

Kinda cool, right? Also notice that Whataburger (a big Texas burger chain) is missing from this dataset because `are.containing('Burger')` is case sensitive.

## Bonus: World Population

Choropleth maps are also for mapping countries across the world. As an example, let's color a map based on the population of each country.

In [ ]:
world_population = Table.read_table('data/world_population.csv')
world_population

However, since `px.choropleth()` expects [ISO Alpha-3 codes](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3), we have to get this information from another table.

In [ ]:
country_codes = Table.read_table('data/country_codes.csv').relabeled('English short name lower case', 'name').select('name', 'Alpha-3 code')
country_codes

But this isn't an issue because we can join tables!

In [ ]:
countries_with_code = world_population.join('name', country_codes)
countries_with_code

Now we're ready to map population by country:

In [ ]:
px.choropleth(countries_with_code.to_df(), 
              locations = 'Alpha-3 code', 
              color = 'pop2022',
              title="World Population (2022) in Thousands",
              color_continuous_scale = 'temps'
             )

The populations of China and India are huge, so it's pretty hard to tell the differences between the populations of other countries. To fix this, we will visualize the populations on the [log_10 scale](https://en.wikipedia.org/wiki/Logarithmic_scale).

In [ ]:
countries_with_code = countries_with_code.with_column(
    'Population (Log)', 
    countries_with_code.apply(np.log10, 'pop2022')
)

In [ ]:
px.choropleth(countries_with_code.to_df(), 
              locations = 'Alpha-3 code', 
              color = 'Population (Log)',
              title="World Population (2022)",
              color_continuous_scale = 'temps'
             )

Viola! A little easier to read.